# 🛰️ SNUNet Change Detection - Kaggle Training

**Before running:**
1. Settings → Accelerator → **GPU T4 x2**
2. Settings → Internet → **On**
3. Add Data → Search **"levir-cd"** → Add dataset

In [ ]:
# === SETUP ===
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if device.type == 'cpu':
    print("⚠️ No GPU! Go to: Settings → Accelerator → GPU T4 x2")

In [ ]:
# === FIND OR CREATE DATASET ===
DATA_ROOT = None

# Search in Kaggle input directories
if os.path.exists('/kaggle/input'):
    for d in os.listdir('/kaggle/input'):
        base = os.path.join('/kaggle/input', d)
        for sub in ['', 'LEVIR-CD', 'levir-cd', 'LEVIR_CD']:
            path = os.path.join(base, sub) if sub else base
            if os.path.exists(path) and os.path.exists(os.path.join(path, 'train')):
                DATA_ROOT = path
                print(f"✅ Found dataset at: {DATA_ROOT}")
                break
        if DATA_ROOT:
            break

# Create synthetic data if no dataset found
if not DATA_ROOT:
    print("❌ Dataset not found. Creating synthetic demo data...")
    DATA_ROOT = '/kaggle/working/data'
    for split in ['train', 'val', 'test']:
        for sub in ['A', 'B', 'label']:
            os.makedirs(os.path.join(DATA_ROOT, split, sub), exist_ok=True)
    
    for split, n in [('train', 200), ('val', 50), ('test', 50)]:
        print(f"  Creating {split}: {n} samples...")
        for i in range(n):
            # Create semi-realistic synthetic change detection data
            base = np.random.randint(50, 200, (256, 256, 3), dtype=np.uint8)
            img1 = Image.fromarray(base)
            img2 = Image.fromarray(base.copy())
            
            # Add random "changed" region
            mask = np.zeros((256, 256), dtype=np.uint8)
            x, y = np.random.randint(20, 200), np.random.randint(20, 200)
            w, h = np.random.randint(20, 50), np.random.randint(20, 50)
            mask[y:y+h, x:x+w] = 255
            
            img1.save(os.path.join(DATA_ROOT, split, 'A', f'{i:04d}.png'))
            img2.save(os.path.join(DATA_ROOT, split, 'B', f'{i:04d}.png'))
            Image.fromarray(mask).save(os.path.join(DATA_ROOT, split, 'label', f'{i:04d}.png'))
    print("✅ Synthetic data created!")

# Show dataset stats
print(f"\nData root: {DATA_ROOT}")
for split in ['train', 'val', 'test']:
    p = os.path.join(DATA_ROOT, split, 'A')
    if os.path.exists(p):
        print(f"  {split}: {len(os.listdir(p))} images")

In [ ]:
# === MODEL ===
class ConvBlock(nn.Module):
    def __init__(self, i, o):
        super().__init__()
        self.c = nn.Sequential(nn.Conv2d(i,o,3,padding=1), nn.BatchNorm2d(o), nn.ReLU(True),
                               nn.Conv2d(o,o,3,padding=1), nn.BatchNorm2d(o), nn.ReLU(True))
    def forward(self, x): return self.c(x)

class Att(nn.Module):
    def __init__(self, c):
        super().__init__()
        self.a = nn.Sequential(nn.AdaptiveAvgPool2d(1), nn.Conv2d(c,c//8,1), nn.ReLU(), nn.Conv2d(c//8,c,1), nn.Sigmoid())
    def forward(self, x): return x * self.a(x)

class SNUNet(nn.Module):
    def __init__(self, c=32):
        super().__init__()
        self.e1,self.e2,self.e3,self.e4,self.e5 = ConvBlock(3,c), ConvBlock(c,c*2), ConvBlock(c*2,c*4), ConvBlock(c*4,c*8), ConvBlock(c*8,c*16)
        self.pool = nn.MaxPool2d(2)
        self.u4,self.d4,self.a4 = nn.ConvTranspose2d(c*16,c*8,2,2), ConvBlock(c*24,c*8), Att(c*8)
        self.u3,self.d3,self.a3 = nn.ConvTranspose2d(c*8,c*4,2,2), ConvBlock(c*12,c*4), Att(c*4)
        self.u2,self.d2,self.a2 = nn.ConvTranspose2d(c*4,c*2,2,2), ConvBlock(c*6,c*2), Att(c*2)
        self.u1,self.d1,self.a1 = nn.ConvTranspose2d(c*2,c,2,2), ConvBlock(c*3,c), Att(c)
        self.out = nn.Conv2d(c,1,1)

    def forward(self, x1, x2):
        e11,e12 = self.e1(x1), self.e1(x2)
        e21,e22 = self.e2(self.pool(e11)), self.e2(self.pool(e12))
        e31,e32 = self.e3(self.pool(e21)), self.e3(self.pool(e22))
        e41,e42 = self.e4(self.pool(e31)), self.e4(self.pool(e32))
        e51,e52 = self.e5(self.pool(e41)), self.e5(self.pool(e42))
        d = torch.abs(e51-e52)
        d = self.a4(self.d4(torch.cat([self.u4(d),e41,e42],1)))
        d = self.a3(self.d3(torch.cat([self.u3(d),e31,e32],1)))
        d = self.a2(self.d2(torch.cat([self.u2(d),e21,e22],1)))
        d = self.a1(self.d1(torch.cat([self.u1(d),e11,e12],1)))
        return self.out(d)

model = SNUNet(32).to(device)
print(f"✅ Model: {sum(p.numel() for p in model.parameters()):,} params")

In [ ]:
# === DATASET ===
class CDData(Dataset):
    def __init__(self, root, split):
        self.root, self.split = root, split
        a_dir = os.path.join(root, split, 'A')
        self.files = sorted([f for f in os.listdir(a_dir) if f.endswith(('.png','.jpg'))])
    def __len__(self): return len(self.files)
    def __getitem__(self, i):
        f = self.files[i]
        im1 = np.array(Image.open(os.path.join(self.root,self.split,'A',f)).convert('RGB').resize((256,256)))
        im2 = np.array(Image.open(os.path.join(self.root,self.split,'B',f)).convert('RGB').resize((256,256)))
        lb = np.array(Image.open(os.path.join(self.root,self.split,'label',f)).convert('L').resize((256,256)))
        return (torch.from_numpy(im1).permute(2,0,1).float()/255,
                torch.from_numpy(im2).permute(2,0,1).float()/255,
                (torch.from_numpy(lb).float()/255>0.5).float().unsqueeze(0))

train_ds = CDData(DATA_ROOT, 'train')
val_split = 'val' if os.path.exists(os.path.join(DATA_ROOT,'val','A')) else 'test'
val_ds = CDData(DATA_ROOT, val_split)
train_ld = DataLoader(train_ds, 8, shuffle=True, num_workers=2, pin_memory=True)
val_ld = DataLoader(val_ds, 16, num_workers=2)
print(f"✅ Train: {len(train_ds)} | Val: {len(val_ds)}")

In [ ]:
# === TRAINING ===
import csv

def loss_fn(p, t):
    bce = F.binary_cross_entropy_with_logits(p, t)
    ps = torch.sigmoid(p)
    dice = 1 - (2*(ps*t).sum()+1)/(ps.sum()+t.sum()+1)
    return bce + dice

def f1_score(p, t):
    p = (torch.sigmoid(p)>0.5).float()
    tp = (p*t).sum(); fp = (p*(1-t)).sum(); fn = ((1-p)*t).sum()
    return (2*tp/(2*tp+fp+fn+1e-8)).item()

EPOCHS = 100
opt = torch.optim.AdamW(model.parameters(), 1e-4, weight_decay=1e-4)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, EPOCHS)

os.makedirs('/kaggle/working/ckpt', exist_ok=True)
log = open('/kaggle/working/ckpt/log.csv','w',newline='')
w = csv.writer(log); w.writerow(['ep','tr_loss','va_loss','f1'])

best, patience = 0, 0
for ep in range(EPOCHS):
    model.train(); tr_loss = 0
    for x1,x2,y in tqdm(train_ld, desc=f"Ep{ep+1}", leave=False):
        x1,x2,y = x1.to(device), x2.to(device), y.to(device)
        opt.zero_grad(); out = model(x1,x2); loss = loss_fn(out,y)
        loss.backward(); torch.nn.utils.clip_grad_norm_(model.parameters(),1); opt.step()
        tr_loss += loss.item()
    tr_loss /= len(train_ld)
    
    model.eval(); va_loss, f1 = 0, 0
    with torch.no_grad():
        for x1,x2,y in val_ld:
            x1,x2,y = x1.to(device), x2.to(device), y.to(device)
            out = model(x1,x2); va_loss += loss_fn(out,y).item(); f1 += f1_score(out,y)
    va_loss /= len(val_ld); f1 /= len(val_ld); sched.step()
    
    print(f"Ep {ep+1:3d} | TrL:{tr_loss:.4f} | VaL:{va_loss:.4f} | F1:{f1:.4f}")
    w.writerow([ep+1,tr_loss,va_loss,f1]); log.flush()
    
    if f1 > best:
        best, patience = f1, 0
        torch.save(model.state_dict(), '/kaggle/working/ckpt/best.pth')
        print(f"   >> Saved (F1={f1:.4f})")
    else:
        patience += 1
        if patience >= 20: print("Early stop!"); break

log.close()
print(f"\n✅ Done! Best F1: {best:.4f}")

In [ ]:
# === VISUALIZATION ===
import matplotlib.pyplot as plt
import pandas as pd

model.load_state_dict(torch.load('/kaggle/working/ckpt/best.pth')); model.eval()

df = pd.read_csv('/kaggle/working/ckpt/log.csv')
fig, ax = plt.subplots(1,2,figsize=(12,4))
ax[0].plot(df['ep'],df['tr_loss'],label='Train'); ax[0].plot(df['ep'],df['va_loss'],label='Val')
ax[0].legend(); ax[0].set_title('Loss'); ax[0].grid()
ax[1].plot(df['ep'],df['f1'],'g-'); ax[1].axhline(0.85,color='r',linestyle='--')
ax[1].set_title(f'F1 (Best:{df.f1.max():.4f})'); ax[1].grid()
plt.tight_layout(); plt.savefig('/kaggle/working/curves.png'); plt.show()

fig, ax = plt.subplots(3,4,figsize=(12,9))
for r in range(3):
    x1,x2,y = val_ds[r*5]
    with torch.no_grad(): p = (torch.sigmoid(model(x1.unsqueeze(0).to(device),x2.unsqueeze(0).to(device)))>0.5).cpu().squeeze().numpy()
    ax[r,0].imshow(x1.permute(1,2,0)); ax[r,1].imshow(x2.permute(1,2,0))
    ax[r,2].imshow(y.squeeze(),cmap='gray'); ax[r,3].imshow(p,cmap='gray')
    for c in range(4): ax[r,c].axis('off')
ax[0,0].set_title('T1'); ax[0,1].set_title('T2'); ax[0,2].set_title('GT'); ax[0,3].set_title('Pred')
plt.tight_layout(); plt.savefig('/kaggle/working/preds.png'); plt.show()
print("\n✅ All done!")